1. join 'volume' and 'trajectories' by 'end_time'
2. have two features: 'count_traj', and 'ave_travel_time'
3. Absolutely need to clean some outliars in 'count'
4. Should consider the delay of conjestion relects on the volume change

1. shift 'trajectories' data by two hours, and then join 'vwgrouped'
2. more than two new featuers
3. split the 'count_traj' by 6 time slots(3 20-min window * 2 hours) and three intersection, which ends to be 18 new features
4. split the 'average_travel_time' as well, another 18 more new features

### 1. volume data during festival needs to be adjusted

In [1]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
pd.set_option('display.max_columns', 500)

import datetime

from __future__ import division

import seaborn as sns

In [2]:
links = pd.read_csv('training/links (table 3).csv')
routes = pd.read_csv('training/routes (table 4).csv')
trajectories = pd.read_csv('training/trajectories(table 5)_training.csv')
volume = pd.read_csv('training/volume(table 6)_training.csv')
weather = pd.read_csv('training/weather (table 7)_training.csv')

submission = pd.read_csv('submission_sample_volume.csv')
test_volume = pd.read_csv('testing_phase1/volume(table 6)_test1.csv')
test_weather = pd.read_csv('testing_phase1/weather (table 7)_test1.csv')

test_trajectories = pd.read_csv('testing_phase1/trajectories(table 5)_test1.csv')

## Convert date

In [3]:
volume.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-09-19 23:09:25,2,0,1,0,NaN
1,2016-09-19 23:11:53,2,0,1,0,NaN
2,2016-09-19 23:13:54,2,0,1,0,NaN
3,2016-09-19 23:17:48,1,0,1,1,NaN
4,2016-09-19 23:16:07,2,0,1,0,NaN


In [4]:
?datetime.datetime.strftime

In [7]:
def date_converter(volume, weather):
    
    # Initialize lists to store splitted information
    date_list = []
    month_list = []
    day_list = []
    hour_list = []
    minute_list = []
    rounded_min_list = []
    rounded_hour_list = []

    # Splits the 'time' information into month, day, hour and minute
    def date_spliter(date):
        
        parts = date.split(" ")
        day_part = parts[0]
        clock_part = parts[1]

        day_parts = day_part.split("-")
        month = int(day_parts[1]) # Extract month from time
        day = int(day_parts[2]) # Extract day from time

        clock_parts = clock_part.split(":")
        hour = int(clock_parts[0]) # Extract hour from time
        minute = int(clock_parts[1]) # Extract minute from time

        rounded_hour = str(hour // 3 * 3)
        rounded_min = str(minute // 20 * 20)

        date_list.append(day_part)
        month_list.append(month)
        day_list.append(day)
        hour_list.append(hour)
        minute_list.append(minute)
        rounded_hour_list.append(rounded_hour)
        rounded_min_list.append(rounded_min)

    # Store info into lists
    for date in volume['time']:
        date_spliter(date)

    # Add arrays into the 'volume' SFrame
    volume['month'] = np.array(month_list)
    volume['day'] = np.array(day_list)
    volume['hour'] = np.array(hour_list)
    volume['minute'] = np.array(minute_list)
    volume['date'] = np.array(date_list)
    volume['rounded_hour'] = np.array(rounded_hour_list)
    volume['rounded_min'] = np.array(rounded_min_list)
    
    # Add an colume which combine 'date' and 'rounded_hour'
    slash_list = np.array(['-'] * len(volume['date']))
    volume['date_and_rounded_hour'] = volume['date'] + slash_list + volume['rounded_hour']
    
    slash_list = np.array(['-'] * len(weather['date']))
    weather['date_and_rounded_hour'] = weather['date'] + slash_list + np.array([str(hour) for hour in weather['hour']])
    
    return (volume, weather)

In [8]:
(volume, weather) = date_converter(volume, weather)

## Clean outliars

In [5]:
# Convert 'wind_direction' outlier to 360
weather.loc[weather['wind_direction'] > 360, 'wind_direction'] = 360

## Build merged table

In [6]:
def build_merged_tables(volume, weather, join_type='inner'):
    # Merge 'volume' and 'weather' DataFrame together
    volume_weather = pd.merge(volume, weather, on='date_and_rounded_hour', suffixes=('', '_y'), how='left')

    # Construct 'window_time' list which uses date, hour, and rounded minute
    date_list = volume_weather['date'] + np.array(['-'] * len(volume_weather))
    hour_list = volume_weather['hour'].astype(str) + np.array(['-'] * len(volume_weather))
    window_time_list = date_list + hour_list + volume_weather['rounded_min'].astype(str)
    volume_weather['window_time'] = window_time_list
    
    return volume_weather

In [7]:
volume_weather = build_merged_tables(volume, weather)

## Group data

In [8]:
# Group 'volume_weather' here
def group_vw(volume_weather):
    
    volume_weather = volume_weather.groupby(['window_time', 'tollgate_id', 'direction'])
    vwgrouped = volume_weather.agg('mean').join(pd.DataFrame(volume_weather.size(), columns=['count']))

    # Put index as column
    vwgrouped['direction'] = vwgrouped.index.get_level_values('direction')
    vwgrouped['tollgate_id'] = vwgrouped.index.get_level_values('tollgate_id')
    vwgrouped['window_time'] = vwgrouped.index.get_level_values('window_time')
    
    vwgrouped['window_time_formatted'] = vwgrouped['window_time'].apply(
        lambda t : datetime.datetime.strptime(t, '%Y-%m-%d-%H-%M'))
    
    return vwgrouped

In [9]:
vwgrouped = group_vw(volume_weather)

## Add features

In [10]:
def re_construct_data(volume_weather):

    # Create 'weekday' column in DateFrame(0 stands for Sunday; 1 stands for Monday and 2 stands for Tuesday, etc...)
    if (9 in list(volume_weather['month'].unique())):
        sept = volume_weather[volume_weather['month'] == 9]
        weekday1 = ((sept['day'] + 3) % 7).values
        octo = volume_weather[volume_weather['month'] == 10]
        weekday2 = ((octo['day'] + 5) % 7).values
        volume_weather['weekday'] = np.append(weekday1, weekday2)
    else:
        weekday2 = ((volume_weather['day'] + 5) % 7).values
        volume_weather['weekday'] = np.array(weekday2)
    
    # Create 'rounded_min' column
    volume_weather['rounded_min'] = volume_weather['minute'] // 20 * 20
    
    # Construct 'is_festival' column
    volume_weather['is_festival'] = np.array([0] * len(volume_weather))
    sep_days = [15, 16, 17]
    oct_days = [1, 2, 3, 4, 5, 6, 7]
    if (9 in list(volume_weather['month'].unique())):
        for day in sep_days:
            volume_weather.loc[((volume_weather['month'] == 9) & (volume_weather['day'] == day)), 'is_festival'] = 1
    for day in oct_days:
        volume_weather.loc[((volume_weather['month'] == 10) & (volume_weather['day'] == day)), 'is_festival'] = 1
        
    # Construct 'is_working_day' column
    volume_weather['is_working_day'] = np.array([0] * len(volume_weather))
    volume_weather.loc[((volume_weather['weekday'] < 5) & (volume_weather['weekday'] > 0)), 'is_working_day'] = 1
    volume_weather.loc[volume_weather['is_festival'] == 1, 'is_working_day'] = 0
    volume_weather.loc[((volume_weather['month'] == 9) & (volume_weather['day'] == 18)), 'is_working_day'] = 1
    volume_weather.loc[((volume_weather['month'] == 10)
                        & ((volume_weather['day'] == 8) | (volume_weather['day'] == 9))), 'is_working_day'] = 1

    
    return volume_weather

In [11]:
vwgrouped = re_construct_data(vwgrouped)

In [12]:
vwgrouped.to_csv('vwgrouped_row.csv')

## Feature engineering

In [12]:
def feature_engineering(vwgrouped):
    # Split numerical data
    vwgrouped['wind_direction'] = vwgrouped['wind_direction'] // 15 * 15
    vwgrouped['precipitation'] = vwgrouped['precipitation'] // 1 * 1
    vwgrouped['wind_speed'] = vwgrouped['wind_speed'] // 1 * 1
    vwgrouped['temperature'] = vwgrouped['temperature'] // 2 * 2
    vwgrouped['rel_humidity'] = vwgrouped['rel_humidity'] // 10 * 10
    
    # Create dummies
    day_list = vwgrouped['day']
    hour_list = vwgrouped['hour']
    vwgrouped = pd.get_dummies(data=vwgrouped, columns=['weekday', 'day', 'hour', 'rounded_min', 'wind_direction',
                                                        'wind_speed', 'temperature', 'rel_humidity', 'precipitation'])
    vwgrouped['day'] = day_list
    vwgrouped['hour'] = hour_list
    
    return vwgrouped

In [13]:
vwgrouped = feature_engineering(vwgrouped)

# Handle trajectories

In [14]:
trajectories = pd.read_csv('training/trajectories(table 5)_training.csv')

## Build and convert time

In [15]:
def time_handler_traj(trajectories):

    # Initialize 'direction' as 0
    trajectories['direction'] = np.array([0] * len(trajectories))
    # Drop milesecond and reformat second
    trajectories['travel_time'] = (trajectories['travel_time'] // 1 * 1).astype(int)
    # Reformat 'starting_time'
    trajectories['starting_time'] = trajectories['starting_time'].apply(
        lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')).tolist()
    # Reformat 'travel_time'
    trajectories['travel_time_formatted'] = trajectories['travel_time'].apply(lambda s : datetime.timedelta(seconds=s)).tolist()
    # Add 'end_time' 
    trajectories['end_time'] = (trajectories['starting_time'] + trajectories['travel_time_formatted']).tolist()

    # Create time features
    e_year_list = trajectories['end_time'].apply(lambda t : t.year).astype(str)
    e_month_list = trajectories['end_time'].apply(lambda t : t.month).astype(str)
    e_day_list = trajectories['end_time'].apply(lambda t : t.day).astype(str)
    e_hour_list = trajectories['end_time'].apply(lambda t : t.hour).astype(str)
    e_min_list = trajectories['end_time'].apply(lambda t : t.minute // 20 * 20).astype(str)
    slash_list = np.array(['-'] * len(trajectories))

    # Create 'window_time'
    trajectories['window_time'] = e_year_list + slash_list + e_month_list + slash_list \
            + e_day_list + slash_list + e_hour_list + slash_list + e_min_list
        
    trajectories['window_time_formatted'] = trajectories['window_time'].apply(
            lambda t : datetime.datetime.strptime(t, '%Y-%m-%d-%H-%M'))
    
    # Shift window_time by 2 hours here
    trajectories['window_time_formatted'] = trajectories['window_time_formatted'].apply(
            lambda t : (t + datetime.timedelta(hours=2)))

    return trajectories

In [16]:
trajectories = time_handler_traj(trajectories)

## Group trajectories

In [17]:
def group_traj(trajectories):
    pre_tra = trajectories.groupby(['window_time_formatted', 'tollgate_id', 'direction'])
    gp_traj = pre_tra.agg('mean').join(pd.DataFrame(pre_tra.size(), columns=['count']))

    gp_traj['window_time_formatted'] = gp_traj.index.get_level_values('window_time_formatted')

    gp_traj.rename(columns={'count':'count_traj'}, inplace=True)
    
    # Put index as column
    gp_traj['direction'] = gp_traj.index.get_level_values('direction')
    gp_traj['tollgate_id'] = gp_traj.index.get_level_values('tollgate_id')
    return gp_traj

In [18]:
gp_traj = group_traj(trajectories)

In [31]:
print sum(gp_traj['count_traj'])

109244


In [32]:
print len(test_trajectories)

2336


In [19]:
def create_combi(vwgrouped, gp_traj):
    
    # Split entry and exit data
    vwgrouped_in = vwgrouped[vwgrouped['direction'] == 0]
    vwgrouped_out = vwgrouped[vwgrouped['direction'] == 1]
    
    # Create combi table
    combi_in = pd.merge(vwgrouped_in, gp_traj, on=['window_time_formatted', 'tollgate_id', 'direction'], how='left')
    combi_out = vwgrouped_out.copy()

    combi_in = combi_in.fillna(combi_in[:combi_in.shape[0]].mean())
    return (combi_in, combi_out)

In [20]:
(combi_in, combi_out) = create_combi(vwgrouped, gp_traj)

In [21]:
# vwgrouped_in = vwgrouped[vwgrouped['direction'] == 0]
# vwgrouped_out = vwgrouped[vwgrouped['direction'] == 1]

In [22]:
# combi_in = pd.merge(vwgrouped_in, gp_traj, on=['window_time_formatted', 'tollgate_id', 'direction'], how='left')
# combi_out = vwgrouped_out.copy()

# combi_in = combi_in.fillna(combi_in[:combi_in.shape[0]].mean())

In [23]:
combi_in.to_csv("combi_in.csv")
combi_out.to_csv("combi_out.csv")

## Create test data

In [24]:
test_volume = pd.read_csv('testing_phase1/volume(table 6)_test1.csv')
test_weather = pd.read_csv('testing_phase1/weather (table 7)_test1.csv')

test_trajectories = pd.read_csv('testing_phase1/trajectories(table 5)_test1.csv')

In [25]:
(test_volume, test_weather) = date_converter(test_volume, test_weather)
# Convert 'wind_direction' outlier to 360
test_weather.loc[test_weather['wind_direction'] > 360, 'wind_direction'] = 360
test_volume_weather = build_merged_tables(test_volume, test_weather)
test_vwgrouped = group_vw(test_volume_weather)
test_vwgrouped = re_construct_data(test_vwgrouped)
test_vwgrouped = feature_engineering(test_vwgrouped)
test_trajectories = time_handler_traj(test_trajectories)
test_gp_traj = group_traj(test_trajectories)
(test_combi_in, test_combi_out) = create_combi(test_vwgrouped, test_gp_traj)
test_combi_in.to_csv("test_combi_in.csv")
test_combi_out.to_csv("test_combi_out.csv")

## Create submission data

In [26]:
submission = pd.read_csv('submission_sample_volume.csv')

sub_weather = pd.read_csv('testing_phase1/weather (table 7)_test1.csv')

sub_trajectories = pd.read_csv('testing_phase1/trajectories(table 5)_test1.csv')

In [27]:
def submission_time_converter(df):
    df['time_formatted'] = df['time_window'].apply(
    lambda t : datetime.datetime.strptime(t.split(',')[0], '[%Y-%m-%d %H:%M:%S'))
    df['year'] = df['time_formatted'].apply(lambda t : t.year).astype(int)
    df['month'] = df['time_formatted'].apply(lambda t : t.month).astype(int)
    df['day'] = df['time_formatted'].apply(lambda t : t.day).astype(int)
    df['hour'] = df['time_formatted'].apply(lambda t : t.hour).astype(int)
    df['rounded_hour'] = ((df['hour'].astype(int)) // 3 * 3).astype(int)
    df['minute'] = df['time_formatted'].apply(lambda t : t.minute).astype(int)
    df['rounded_min'] = (df['minute'].astype(int) // 20 * 20).astype(int)
    slash = np.array(['-'] * len(df))
    df['date_and_rounded_hour'] = df['year'].astype(str) + slash + df['month'].astype(str) \
                                    + slash + df['day'].astype(str) + slash + df['rounded_hour'].astype(str)
    return df

In [28]:
sub_weather['date_and_rounded_hour'] = sub_weather['date'] + np.array(['-'] * len(sub_weather)) + sub_weather['hour'].astype(str)

In [29]:
sub_volume = submission_time_converter(submission)

In [30]:
sub_volume_weather = build_merged_tables(sub_volume, sub_weather)
sub_vwgrouped = group_vw(sub_volume_weather)
sub_vwgrouped = re_construct_data(sub_vwgrouped)
sub_vwgrouped = feature_engineering(sub_vwgrouped)
sub_trajectories = time_handler_traj(sub_trajectories)
sub_gp_traj = group_traj(sub_trajectories)
(sub_combi_in, sub_combi_out) = create_combi(sub_vwgrouped, sub_gp_traj)

sub_combi_in.to_csv("sub_combi_in.csv")
sub_combi_out.to_csv("sub_combi_out.csv")
sub_gp_traj.to_csv("sub_gp_traj.csv")